In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import KernelPCA
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [4]:
# ---------------------------
# Functions
# ---------------------------
def Func_KPCA(X_train, X_test):
    kpca = KernelPCA(n_components=2, kernel='rbf', gamma=0.1, random_state=0)
    X_train_kpca = kpca.fit_transform(X_train)
    X_test_kpca = kpca.transform(X_test)
    return kpca, X_train_kpca, X_test_kpca

def evaluate_model(name, classifier, X_train, y_train, X_test, y_test):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n=== {name} ===")
    print("Accuracy:", acc)
    print("Confusion Matrix:\n", cm)
    return acc, cm

In [5]:
# ---------------------------
# Load dataset
# ---------------------------
dataset = pd.read_csv("Wine.csv")
X = dataset.iloc[:, 0:13].values
y = dataset.iloc[:, 13].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Apply Kernel PCA
kpca, X_train_kpca, X_test_kpca = Func_KPCA(X_train, X_test)

In [6]:
# ---------------------------
# Train classifiers on KPCA data
# ---------------------------
results = {}

results["Logistic"] = evaluate_model("Logistic Regression", LogisticRegression(random_state=0), 
                                     X_train_kpca, y_train, X_test_kpca, y_test)

results["SVM (linear)"] = evaluate_model("SVM (linear)", SVC(kernel='linear', random_state=0), 
                                         X_train_kpca, y_train, X_test_kpca, y_test)

results["SVM (rbf)"] = evaluate_model("SVM (rbf)", SVC(kernel='rbf', random_state=0), 
                                      X_train_kpca, y_train, X_test_kpca, y_test)

results["KNN"] = evaluate_model("KNN", KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2), 
                                X_train_kpca, y_train, X_test_kpca, y_test)

results["Naive Bayes"] = evaluate_model("Naive Bayes", GaussianNB(), 
                                        X_train_kpca, y_train, X_test_kpca, y_test)

results["Decision Tree"] = evaluate_model("Decision Tree", DecisionTreeClassifier(criterion='entropy', random_state=0), 
                                          X_train_kpca, y_train, X_test_kpca, y_test)

results["Random Forest"] = evaluate_model("Random Forest", RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0), 
                                          X_train_kpca, y_train, X_test_kpca, y_test)



=== Logistic Regression ===
Accuracy: 1.0
Confusion Matrix:
 [[14  0  0]
 [ 0 16  0]
 [ 0  0  6]]

=== SVM (linear) ===
Accuracy: 1.0
Confusion Matrix:
 [[14  0  0]
 [ 0 16  0]
 [ 0  0  6]]

=== SVM (rbf) ===
Accuracy: 1.0
Confusion Matrix:
 [[14  0  0]
 [ 0 16  0]
 [ 0  0  6]]

=== KNN ===
Accuracy: 1.0
Confusion Matrix:
 [[14  0  0]
 [ 0 16  0]
 [ 0  0  6]]

=== Naive Bayes ===
Accuracy: 1.0
Confusion Matrix:
 [[14  0  0]
 [ 0 16  0]
 [ 0  0  6]]

=== Decision Tree ===
Accuracy: 0.9722222222222222
Confusion Matrix:
 [[14  0  0]
 [ 1 15  0]
 [ 0  0  6]]

=== Random Forest ===
Accuracy: 0.9722222222222222
Confusion Matrix:
 [[14  0  0]
 [ 1 15  0]
 [ 0  0  6]]


In [7]:
# ---------------------------
# Build summary table (only accuracies)
# ---------------------------
summary = {name: acc for name, (acc, cm) in results.items()}
result_df = pd.DataFrame(summary, index=["KernelPCA"])
print("\n\n=== Summary Table (Accuracies) ===")
print(result_df)



=== Summary Table (Accuracies) ===
           Logistic  SVM (linear)  SVM (rbf)  KNN  Naive Bayes  Decision Tree  \
KernelPCA       1.0           1.0        1.0  1.0          1.0       0.972222   

           Random Forest  
KernelPCA       0.972222  
